<a href="https://colab.research.google.com/github/GerardoMunoz/PresentacionesAlgebraLineal/blob/main/PS_angulo_word_vect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En la página https://matematicaj.blogspot.com/2020/03/Test-de-analogias-verbales-resuelto-con-claves-y-respuestas-pre-universidad-pdf.html se presentan algunas analogías de palabras. En el primer caso se pregunta soldado es a ejército como:
1. Ratón a gato,
2. Llave a llavero,
3. Paradoja a contradicción,
4. Pulga a perro,
5. Tornillo a máquina.

Usaremos el álgebra lineal para resolver el problema. A principio de semestre vimos como  algoritmos (como bolsa de palabras, Word2vec, Gensim, GloVe) aplicado a un conjunto de documentos (corpus), [incrustan las palabras](https://unipython.com/como-desarrollar-embeddings-incrustaciones-de-palabras-con-gensim/) <!--(https://es.wikipedia.org/wiki/Word_embedding)--> en $\mathbb{R}^n$.  

Algunas de estas incrustaciones tienen propiedades semánticas, es decir que algunas operaciones vectoriales son congruentes con el significado de las palabras. Como en el famoso ejemplo  Rey - Hombre + Mujer -> Reina. 

Para este ejercicio vamos a utilizar la librería `spacy` con una base de datos de noticias (news) pequeña (sm) en español (es) llamada `es_core_news_sm`. 

* Lo primero es obtener el vector $\vec{u}_0$  para la palabra soldado y el vector $\vec{v}_0$ para la palabra ejército. Los vectores no están ni en $\mathbb{R}^2$ ni en $\mathbb{R}^3$ sino en $\mathbb{R}^{96}$

* Luego se obtiene el vector  $\vec{w}_0 =  \vec{u}_0 - \vec{v}_0$ que va desde soldado hasta ejército.

* De manera similar se obtiene el vector para la primera repuesta. Es decir, el vector $\vec{w}_1$ que va de raton a gato.  Finalmente se calcula el coseno del ángulo $\cos(\alpha_1)$ entre el vector  $\vec{w}_0$ y $\vec{w}_1$.

* Se repite los mismo para cada par de palabras, obteniendo los vectores $\vec{w}_2$, $\vec{w}_3$, $\vec{w}_4$, $\vec{w}_5$. Posteriormente los cosenos  $\cos(\alpha_2)$, $\cos(\alpha_3)$, $\cos(\alpha_4)$, $\cos(\alpha_5)$, entre los vectores  $\vec{w}_0$ y el respectivo $\vec{w}_i$.

*  Finalmente se selecciona la respuesta con el mayor coseno del ángulo.

Lo primero es descargar la librería en español "es_core_news_sm". Esto sólo hay que realizarlo una vez se inicia. 



In [ ]:
!python -m spacy download es_core_news_sm

     |████████████████████████████████| 16.2 MB 5.0 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')


En Colab puede 'Reinicar el entorno de ejecución' despues de descargar la librería 

Lo primero es llamar la libreria spacy. 

In [ ]:
import spacy

Luego sigue cargar los datos del español que se descargaron previamente. En caso de no haber reiniciado el entorno de ejecución puede que se presente un error.

In [ ]:
spacy_es = spacy.load("es_core_news_sm")

Ahora si comenzamos a obtener los vectores de cada palabra. Es importante que la palabra se escriba con tilde si las tiene. 

In [ ]:
soldado=spacy_es("soldado").vector
soldado

array([ 6.3344855 ,  7.8538084 , -3.833856  ,  0.8017853 ,  4.647547  ,
       -3.6115787 , -7.9624004 , -4.3260913 , -0.6534754 , -0.34920466,
       -1.0873322 , -4.438015  , -8.063329  ,  7.348127  ,  1.0255387 ,
        1.6311729 , -0.4751122 ,  4.8554654 , -2.895939  ,  5.538233  ,
        1.1452105 , -6.394264  ,  0.83366513, -1.2777112 ,  3.8527415 ,
       -3.4849334 , -7.7172914 ,  0.19821191, -0.42739558, -1.6300942 ,
       -3.101515  , -4.615014  ,  3.6517115 , -2.6606016 ,  3.3364868 ,
       -3.8454266 ,  1.9919299 , -1.3486339 ,  2.4598174 ,  2.6024532 ,
       -0.7598468 , -0.5252705 ,  3.7853131 ,  4.3498306 , -1.8674426 ,
        0.9362514 , -1.3034072 , -5.7030444 ,  1.3807571 , -5.042339  ,
       -1.9307297 ,  2.250323  ,  2.0519018 , -3.100582  ,  3.2007494 ,
        4.7060933 , -0.99164975,  4.880626  ,  0.73361194,  0.87203723,
       -2.3065236 ,  5.947562  , -0.47327584, -0.6609935 ,  1.0311747 ,
        4.1003036 ,  1.6786666 , -3.324019  ,  1.7406416 ,  0.50

Observe que cada palabra es un vector de 96 elementos

In [ ]:
len(soldado)

96

In [ ]:
ejercito=spacy_es("ejército").vector
ejercito

array([ 7.538803  ,  8.917214  , -5.756281  , -2.04584   ,  5.580385  ,
       -3.626072  , -7.449911  , -0.5357484 ,  2.2351847 ,  4.995864  ,
       -0.50550306,  1.3766865 , -4.196176  ,  1.7708406 , -1.1264827 ,
        1.8160542 , -2.6417944 ,  0.9113256 , -1.718248  ,  4.3655376 ,
       -0.19247319, -6.596141  ,  2.5170288 , -2.2334785 ,  1.3446064 ,
       -5.6206713 , -2.7808964 , -1.702582  , -0.88280857, -5.781607  ,
       -3.1784806 , -4.5840597 ,  1.4608195 , -6.605699  ,  3.5741682 ,
       -3.7378194 ,  2.9579155 , -2.3093266 ,  2.825917  ,  3.0586195 ,
       -3.0983527 ,  1.1624153 , -2.3209682 ,  0.22037837,  0.50732464,
       -0.20533124, -2.6595716 , -0.11621785,  4.7312164 , -2.7816224 ,
        0.86406237, -0.5964475 ,  2.7367587 , -4.208027  , -4.3122168 ,
        1.8786385 ,  0.87293446,  7.759996  ,  0.0966503 ,  2.1353645 ,
        4.2650576 ,  5.1819644 , -1.8039317 ,  1.1279613 ,  4.182113  ,
       -0.04996499,  2.5031712 , -5.781842  ,  1.468445  ,  0.26

In [ ]:
soldado_ejercito = soldado - ejercito
soldado_ejercito

array([-1.2043176 , -1.063406  ,  1.9224248 ,  2.8476253 , -0.93283844,
        0.01449323, -0.5124893 , -3.7903428 , -2.88866   , -5.3450685 ,
       -0.5818292 , -5.8147016 , -3.8671527 ,  5.5772862 ,  2.1520214 ,
       -0.18488133,  2.1666822 ,  3.94414   , -1.1776911 ,  1.1726952 ,
        1.3376837 ,  0.20187664, -1.6833637 ,  0.9557674 ,  2.508135  ,
        2.135738  , -4.9363947 ,  1.9007939 ,  0.45541298,  4.151513  ,
        0.07696557, -0.03095436,  2.190892  ,  3.9450974 , -0.23768139,
       -0.10760713, -0.96598566,  0.96069276, -0.3660996 , -0.45616627,
        2.3385057 , -1.6876857 ,  6.1062813 ,  4.129452  , -2.3747673 ,
        1.1415826 ,  1.3561645 , -5.5868263 , -3.3504593 , -2.2607164 ,
       -2.7947922 ,  2.8467705 , -0.6848569 ,  1.107445  ,  7.512966  ,
        2.8274548 , -1.8645842 , -2.8793697 ,  0.63696164, -1.2633274 ,
       -6.571581  ,  0.7655978 ,  1.3306558 , -1.7889547 , -3.1509385 ,
        4.1502686 , -0.8245046 ,  2.4578233 ,  0.27219665,  0.24

In [ ]:
raton = spacy_es("ratón").vector
gato = spacy_es("gato").vector
raton_gato = raton - gato
raton_gato

array([-7.8128619e+00, -5.1120386e+00, -2.8030500e+00, -1.4234204e+00,
        3.3935843e+00,  6.7414804e+00,  3.3021860e+00, -3.1290128e+00,
       -3.0188837e+00,  1.2931148e+00, -4.2101846e+00,  1.3095536e+00,
       -1.6784016e+00,  2.3608499e+00, -2.2683332e+00,  1.2097616e+00,
       -2.3473423e+00,  1.2047687e+00,  4.0310316e+00, -3.3584726e+00,
        1.0980959e+00,  1.1955526e+00,  1.0988231e+00,  6.7072862e-01,
        2.7143583e+00, -2.1168501e+00, -5.7886901e+00,  3.6043692e+00,
       -7.5329643e-01, -3.8307872e+00,  3.1315994e-01,  2.8146472e+00,
       -1.7964878e+00, -2.7211072e+00,  3.7243035e+00,  3.2367647e-02,
        1.4622737e+00,  5.4436147e-01,  4.9615226e+00, -4.5427138e-01,
       -2.6951001e+00, -7.7229738e-04, -2.8688343e+00, -8.8544869e-01,
       -5.5224299e-02,  3.6975425e-01, -1.3796983e+00,  2.8418670e+00,
       -3.2898965e+00, -4.4440942e+00,  1.2491769e+00, -3.1466155e+00,
        2.6476614e+00, -1.6129951e+00,  1.6012613e+00, -3.3363862e+00,
      

In [ ]:
import numpy as np
# Los vectores obtenidos son arreglos de numpy
# El producto punto entre dos vectores se puede realizar con el método `dot`
# La norma de un vector se realiza con la función `np.linalg.norm`

In [ ]:
cos_preg_rta1 = soldado_ejercito.dot(raton_gato) / (np.linalg.norm(soldado_ejercito) * np.linalg.norm(raton_gato))
cos_preg_rta1

-0.087970614

In [ ]:
llave = spacy_es("llave").vector
llavero = spacy_es("llavero").vector
llave_llavero = llave - llavero

cos_preg_rta2 = soldado_ejercito.dot(llave_llavero) / (np.linalg.norm(soldado_ejercito) * np.linalg.norm(llave_llavero))
cos_preg_rta2

-0.14314723

In [ ]:
paradoja = spacy_es("paradoja").vector
contradiccion = spacy_es("contradicción").vector
paradoja_contradiccion = paradoja - contradiccion

cos_preg_rta3 = soldado_ejercito.dot(paradoja_contradiccion) / (np.linalg.norm(soldado_ejercito) * np.linalg.norm(paradoja_contradiccion))
cos_preg_rta3

-0.06602139

In [ ]:
pulga = spacy_es("pulga").vector
perro = spacy_es("perro").vector
pulga_perro = pulga - perro

cos_preg_rta4 = soldado_ejercito.dot(pulga_perro) / (np.linalg.norm(soldado_ejercito) * np.linalg.norm(pulga_perro))
cos_preg_rta4

0.2551043

In [ ]:
tornillo = spacy_es("tornillo").vector
maquina = spacy_es("máquina").vector
tornillo_maquina = tornillo - maquina

cos_preg_rta5 = soldado_ejercito.dot(tornillo_maquina) / (np.linalg.norm(soldado_ejercito) * np.linalg.norm(tornillo_maquina))
cos_preg_rta5

0.0015725981

Coseno del ángulo con soldado ejercito   

| palabra1 | palabra2    |        |
|----------|-------------|---------------|
|ratón     |gato         |-0.087970614   |
|llave     |llavero      |-0.14314723    |
|paradoja  |contradicción|-0.06602139    |
|pulga     |perro        | 0.2551043     |
|tornillo  |máquina      | 0.0015725981  |

La mejor respuesta es la que el valor absoluto del coseno del ángulo sea mayor. En este caso sería pulga-perro, pero no corresponde a la respuesta correcta. 

La respuesta correcta es llave-llavero, que obtuvo un segundo lugar. Que al igual que soldado-ejército,  tienen una relación de elemento - conjunto.

Sin embargo, si tenemos en cuenta que tradicionalmente se ha asociado al perro con muchas pulgas, podría sugerirse esa como una causa de la respuesta.  


Por otra parte, sabemos que una máquina usualmente tiene muchos tornillos, pero el bajo valor del coseno seguramente se debe a que pocas veces se habla al respecto. 
